##Малышева Екатерина, 18ПМИ


Устанавливаем зависимости 



In [1]:
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode


## Install NeMo
!python -m pip install nemo_toolkit[all]

# !pip install torchtext==0.10.1
# !pip install matplotlib==3.1.3
"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""

! rm -rf sample_data
exit()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [5]:
from google.colab import drive
import os
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [6]:
data_dir = '/drive/MyDrive/NIS_Lab2/'

На диске расположены 2 манифеста - для farfield_train и farfield_test в следующем виде: 

NIS_Lab2/
* farfield_test/
   * manifest.jsonl
   * files/
* farfield_train/
    * manifest_all_train.jsonl

Также файлы для тестов находятся в папке files, позднее на диск еще будут добавлены аудиофайлы для тренировки.

Преобразуем manifest_all_train.jsonl (уберем референсы неиспользуемых файлов)

In [7]:
import json
import glob
import os
import subprocess
import tarfile
import wget
from shutil import copyfile
def clean_man(path_to_manifest):
  file_to = os.path.join(path_to_manifest, 'manifest.jsonl')
  file_from = os.path.join(path_to_manifest, 'manifest_all_train.jsonl')
  with open(file_from, 'r') as ff:
    with open(file_to, 'w+') as ft:
      for line in ff:
        metadata = json.loads(line)
        if metadata["audio_filepath"].startswith("farfield"):
          json.dump(metadata, ft)
          ft.write('\n')
clean_man('/drive/MyDrive/NIS_Lab2/farfield_train/')

Затем перепишем пути в манифестах на абсолютные.

In [8]:
# json with absolute paths

def build_manifest_abs(path_to_manifest):
  not_missing =  0
  missing = 0
  file_to = os.path.join(path_to_manifest, 'manifest_abs.jsonl')
  file_from = os.path.join(path_to_manifest, 'manifest.jsonl')
  with open(file_from, 'r') as ff:
    with open(file_to, 'w+') as ft:
      for line in ff:
        metadata = json.loads(line)
        if path_to_manifest == '/drive/MyDrive/NIS_Lab2/farfield_train':
          metadata["audio_filepath"]=os.path.join('/drive/MyDrive/NIS_Lab2/farfield_train/train/', metadata["audio_filepath"])
        else:
          metadata["audio_filepath"]=os.path.join(path_to_manifest, metadata["audio_filepath"])
        if os.path.exists(metadata["audio_filepath"]):
          json.dump(metadata, ft)
          ft.write('\n')
          not_missing +=1
        else:
          missing +=1 
  print('missing paths', missing)    
  print('not missing paths', not_missing)
  return file_to

manifest_test_abs = build_manifest_abs('/drive/MyDrive/NIS_Lab2/farfield_test' )
print(manifest_test_abs)
manifest_train_abs = build_manifest_abs('/drive/MyDrive/NIS_Lab2/farfield_train')
print(manifest_train_abs )

missing paths 0
not missing paths 1916
/drive/MyDrive/NIS_Lab2/farfield_test/manifest_abs.jsonl
missing paths 32816
not missing paths 91187
/drive/MyDrive/NIS_Lab2/farfield_train/manifest_abs.jsonl


Теперь загрузим farfield train data в /drive/MyDrive/NIS_Lab2/farfield_train/train/farfield

In [9]:
def load(golos_url, fname, data_dir):
  if not os.path.exists(os.path.join(data_dir, fname)):
    print('Downloading', data_dir)
    golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
    print(f"Dataset downloaded at: {golos_path}")
  else:
      print("File already exists.")
      golos_path = os.path.join(data_dir, fname)
  return golos_path

if not os.path.exists(os.path.join(data_dir, 'tar')):
  os.mkdir('tar') 
golos_path = load("https://sc.link/1Z3", "train_farfield.tar", os.path.join(data_dir, 'tar') )
print('tar path:', golos_path)
if not os.path.exists(os.path.join(data_dir, 'farfield_train/')):
  print('creating farfield_train/')
  os.mkdir(os.path.join(data_dir, 'farfield_train/'))
if not os.path.exists(os.path.join(data_dir, 'farfield_train/train')):
  print('files not extracted') 
  with tarfile.open(golos_path) as tar:
    print(f"Extracting train...")
    tar.extractall(path=os.path.join(data_dir, 'farfield_train'))
    print("Done")


File already exists.
tar path: /drive/MyDrive/NIS_Lab2/tar/train_farfield.tar


In [10]:
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr

[NeMo W 2021-11-30 17:05:57 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



## Использование предобученной модели


Коллекция для распознавания речи в NeMo содержит готовые блоки, которые можно использовать чтобы тренировать и использовать свою модель. Кроме этого существет ряд предобученных моделей, которые можно просто скачать и исползовать. Давайте скачаем и инициализируем готовую модель QuartzNet15x5, обученную на открытом датасете Golos.



In [11]:
print(data_dir)
os.listdir(data_dir) 

/drive/MyDrive/NIS_Lab2/


['farfield_test', 'tar', 'farfield_train', 'QuartzNet15x5_golos.nemo']

In [12]:
load("https://sc.link/ZMv", "QuartzNet15x5_golos.nemo", data_dir)
asr_model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"))

[NeMo I 2021-11-30 17:06:05 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /drive/MyDrive/NIS_Lab2/QuartzNet15x5_golos.nemo.


Теперь указваем список фалов которые мы хотим транскрибировать и передаем в нашу модель. Это будет работать для относительно коротких аудио (<25 секунд) файлов. 

In [13]:
files = [ "/drive/MyDrive/NIS_Lab2/farfield_test/files/0a241e2a814d5895808c135eebcc48e3.wav",
          "/drive/MyDrive/NIS_Lab2/farfield_test/files/0af12c9f57213f06c2b2f59817038df8.wav"]
transcriptions = asr_model.transcribe(paths2audio_files=files)
for fname, transcription in zip(files, transcriptions):
  print(f"Audio in {fname} was recognized as: {transcription}")

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2021-11-30 17:06:09 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2021-11-30 17:06:09 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


Audio in /drive/MyDrive/NIS_Lab2/farfield_test/files/0a241e2a814d5895808c135eebcc48e3.wav was recognized as: афина почему трампы трамвай едут поарелзам
Audio in /drive/MyDrive/NIS_Lab2/farfield_test/files/0af12c9f57213f06c2b2f59817038df8.wav was recognized as: салют покажи телеканал три ангела


Устанавливаем в конфигурацию предобученной модели нужные наборы данных (test и train)

In [14]:
from omegaconf import DictConfig
params_model = asr_model._cfg

params_model['train_ds']['parser']='base'
params_model['validation_ds']['parser']='base'
params_model['test_ds']['parser']='base'
params_model['train_ds']['manifest_filepath'] = manifest_train_abs
params_model['train_ds']['batch_size']=16
params_model['validation_ds']['manifest_filepath'] = manifest_test_abs
params_model['validation_ds']['batch_size']=16
params_model['test_ds']['manifest_filepath'] = manifest_test_abs
params_model['test_ds']['batch_size']=16
params_model['optim']['lr'] = 0.05

asr_model.setup_optimization(optim_config=DictConfig(params_model['optim']))
#asr_model.setup_training_data(train_data_config=params_model['train_ds'])
asr_model.setup_test_data(test_data_config=params_model['test_ds'])
asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])

[NeMo W 2021-11-30 17:06:15 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-30 17:06:15 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.05
        weight_decay: 0.001
    )


[NeMo W 2021-11-30 17:06:15 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2021-11-30 17:06:20 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 17:06:20 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 17:06:20 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 17:06:20 collections:174] 0 files were filtered totalling 0.00 hours


[NeMo W 2021-11-30 17:06:20 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


WER для предообученной модели, тестируем на farfield_test/files

In [15]:
asr_model.cuda()
wer_nums = []
wer_denoms = []

for test_batch in asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = asr_model._wer.compute()
        asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.

print(sum(wer_nums), sum(wer_denoms))
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo W 2021-11-30 17:06:26 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


5527.0 8382.0
WER = 0.659389167263183


### Использование PyTorch Lightning

#### Первая попытка дообучения


In [16]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1, max_epochs=2, precision = 16)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Модифицируем параметры

In [17]:
first_asr_model = asr_model
import copy
new_opt = copy.deepcopy(params_model['optim'])
new_opt['lr'] = 0.001
first_asr_model.setup_optimization(optim_config=DictConfig(new_opt))
first_asr_model.setup_training_data(train_data_config=params_model['train_ds'])
first_asr_model.setup_test_data(test_data_config=params_model['test_ds'])
first_asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])

[NeMo W 2021-11-30 17:06:51 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-30 17:06:51 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )


[NeMo W 2021-11-30 17:06:51 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2021-11-30 17:06:55 collections:173] Dataset loaded with 91181 files totalling 100.79 hours
[NeMo I 2021-11-30 17:06:55 collections:174] 6 files were filtered totalling 0.02 hours


[NeMo W 2021-11-30 17:06:55 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-30 17:06:55 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 17:06:55 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 17:06:55 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 17:06:55 collections:174] 0 files were filtered totalling 0.00 hours


Обучаем 

In [18]:
# Start training!!!
trainer.fit(first_asr_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-30 17:07:03 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-30 17:07:03 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.001
        weight_decay: 0.001
    )


[NeMo W 2021-11-30 17:07:03 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
37.859    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 17:07:03 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    
[NeMo W 2021-11-30 17:07:05 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [19]:
first_asr_model.save_to('/drive/MyDrive/NIS_Lab2/first_asr_model.nemo')


In [22]:
print(first_asr_model.transcribe(paths2audio_files=files,
                                 batch_size=4))

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2021-11-30 19:21:27 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


['афина почему крампайтомвай вегпарец', 'салют покажи телеканал три ангела']


#### WER для первой модели 

In [21]:
# Bigger batch-size = bigger throughput

# Setup the test data loader and make sure the model is on GPU
first_asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in first_asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = first_asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        first_asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = first_asr_model._wer.compute()
        first_asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo W 2021-11-30 19:19:32 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER = 0.6083273681698879


#### Вторая попытка дообучения 

In [23]:
second_asr_model = asr_model
import copy
new_opt = copy.deepcopy(params_model['optim'])
new_opt['lr'] = 0.0001
second_asr_model.setup_optimization(optim_config=DictConfig(new_opt))
second_asr_model.setup_training_data(train_data_config=params_model['train_ds'])
second_asr_model.setup_test_data(test_data_config=params_model['test_ds'])
second_asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])

[NeMo W 2021-11-30 19:32:51 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-30 19:32:51 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )


[NeMo W 2021-11-30 19:32:51 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2021-11-30 19:32:54 collections:173] Dataset loaded with 91181 files totalling 100.79 hours
[NeMo I 2021-11-30 19:32:54 collections:174] 6 files were filtered totalling 0.02 hours


[NeMo W 2021-11-30 19:32:54 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-30 19:32:55 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 19:32:55 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-30 19:32:55 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-30 19:32:55 collections:174] 0 files were filtered totalling 0.00 hours


In [27]:
trainer = pl.Trainer(gpus=1, max_epochs=1, precision = 16)
trainer.fit(second_asr_model)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-30 19:55:10 modelPT:453] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-30 19:55:10 modelPT:561] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )


[NeMo W 2021-11-30 19:55:10 lr_scheduler:732] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
37.859    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-30 19:55:11 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    
[NeMo W 2021-11-30 19:55:13 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [28]:
second_asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in first_asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = first_asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        first_asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = first_asr_model._wer.compute()
        first_asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo W 2021-11-30 20:59:08 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/nemo/collections/asr/parts/submodules/jasper.py:280: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
      ) // self.conv.stride[0] + 1
    


WER = 0.5622763063707945
